In [16]:
import numpy as np
# S: State Value of house : Its probability of retaining kings landing
# P: Transition Matrix : from houseA to houseB, what is probability of transistion.

houses = ["lannisters","starks"]
S = {"lannisters": 0.1, "starks":0.9}
P = {"from_lannisters":{"to_lannisters": 0.8, "to_starks" : 0.2 }, 
     "from_starks": {"to_starks": 0.1, "to_lannisters": 0.9}}


for i in range(5):
    lannisters_future_stateval, starks_future_stateval = 0,0;
    
    lannisters_future_stateval = S["lannisters"]*P["from_lannisters"]["to_lannisters"]+\
                                S["starks"]*P["from_starks"]["to_lannisters"];
    
    starks_future_stateval =  S["lannisters"]*P["from_lannisters"]["to_starks"]+\
                                S["starks"]*P["from_starks"]["to_starks"];

    S["lannisters"]=lannisters_future_stateval
    S["starks"] = starks_future_stateval
    print S
    
print S
print "Volla, '%s' have been winners with %0.2f%% probability to be in king's landing "%(max(S, key=S.get),100*max(S.values()))



{'starks': 0.11000000000000001, 'lannisters': 0.8900000000000001}
{'starks': 0.18900000000000006, 'lannisters': 0.8110000000000002}
{'starks': 0.18110000000000004, 'lannisters': 0.8189000000000002}
{'starks': 0.18189000000000005, 'lannisters': 0.8181100000000002}
{'starks': 0.18181100000000006, 'lannisters': 0.8181890000000003}
{'starks': 0.18181100000000006, 'lannisters': 0.8181890000000003}
Volla, 'lannisters' have been winners with 81.82% probability to be in king's landing 


In [1]:
# Markov Reward Process Example
import numpy as np
import copy

S ={"dragonstone": 1,"whiteharbor":1, "winterfell":1,"alive-terminal":-1,"dead-terminal":-1} # States
R = {"dragonstone": 0,"whiteharbor":10, "winterfell":50,"alive-terminal":100,"dead-terminal":-50}
P ={"from_dragonstone":{"to_whiteharbor":0.5,"to_winterfell":0.1,"to_dead-terminal":0.4},\
    "from_whiteharbor":{"to_winterfell":0.1,"to_dead-terminal":0.9},
   "from_winterfell":{"to_alive-terminal":0.9,"to_dead-terminal":0.1},
    "from_alive-terminal":{},
    "from_dead-terminal":{}
   }
gamma = 0.1

for i in range(100):
    S1 = copy.copy(S);
    for castle in S1.keys():
        reward =0;
        from_castle='from_'+castle
        future_states = P[from_castle]
        for to_castle in future_states:
            reward=reward+P[from_castle][to_castle]*S[to_castle[3:]];
        
        S1[castle] = R[castle]+ gamma*(reward) 
    S = copy.copy(S1);
print S



{'whiteharbor': 6.085, 'winterfell': 58.5, 'dragonstone': -1.1107500000000001, 'dead-terminal': -50.0, 'alive-terminal': 100.0}


In [9]:
# Markov Decision Process Example
import numpy as np
import copy

S ={"dragonstone": 1,"whiteharbor":1, "winterfell":1,"alive-terminal":-1,"dead-terminal":-1} # States

R = {"dragonstone": 0,"whiteharbor":10, "winterfell":50,"alive-terminal":100,"dead-terminal":-50}
P ={"from_dragonstone":{"land":{"to_winterfell":0.5,"to_dead-terminal":0.5},\
                         "sea":{"to_whiteharbor":0.1,"to_dead-terminal":0.9},\
                         "dragon":{"to_winterfell":0.95,"to_dead-terminal":0.05}},\
    "from_whiteharbor":{"land":{"to_winterfell":0.6,"to_dead-terminal":0.4}},\
   "from_winterfell":{"land":{"to_alive-terminal":0.9,"to_dead-terminal":0.1}},\
    "from_alive-terminal":{},\
    "from_dead-terminal":{}
   }

gamma = 0.1

Policy = {"from_dragonstone":"land","from_whiteharbor":"land","from_winterfell":"land"}


# Solution by Value Iteration
for i in range(1000):
    S1 = copy.copy(S);
    
    for castle in S1.keys():
        from_castle='from_'+castle
        future_states = P[from_castle]
        action_reward={}
        for action in future_states.keys():
            reward =0;
            for to_castle in future_states[action]:
                reward=reward+P[from_castle][action][to_castle]*S[to_castle[3:]];
            action_reward[action]=reward
        if(len(action_reward.keys())>0): 
            Policy[from_castle]=max(action_reward,key=action_reward.get)
            S1[castle] = R[castle]+gamma*(max(action_reward.values()))
        else:
            S1[castle] = R[castle]
    S = copy.copy(S1);
print 'Value Function',S
print 'Learned Policy',Policy



Value Function {'whiteharbor': 11.51, 'winterfell': 58.5, 'dragonstone': 5.3075, 'dead-terminal': -50, 'alive-terminal': 100}
Learned Policy {'from_whiteharbor': 'land', 'from_dragonstone': 'dragon', 'from_winterfell': 'land'}


# Solution of MDP by Policy Iteration

In [15]:
#Solution by Policy Iteration    
Policy = {"from_dragonstone":"land","from_whiteharbor":"land","from_winterfell":"land"}

for i in range(10):
    #Step 1 : Update Value Fuction
    for j in range(100):
        S1 = copy.copy(S);
        for castle in S1.keys():
            from_castle='from_'+castle
            future_states = P[from_castle]
            action_reward={}
            if (from_castle not in Policy.keys()):
                S1[castle]=R[castle]
                continue
            action = Policy[from_castle] #Based on 
            reward=0;
            for to_castle in future_states[action]:
                reward=reward+P[from_castle][action][to_castle]*S[to_castle[3:]];
            S1[castle] = R[castle]+ gamma*(reward)            
        S = copy.copy(S1);
    #Step 2 : Update Policy
    for castle in S.keys():
        from_castle='from_'+castle
        future_states = P[from_castle]
        action_reward={}
        for action in future_states.keys():
            reward =0;
            for to_castle in future_states[action]:
                reward=reward+P[from_castle][action][to_castle]*S[to_castle[3:]];
            action_reward[action]=reward
        if(len(action_reward.keys())>0):
            Policy[from_castle]=max(action_reward,key=action_reward.get) #Update Policy

    
print 'Value Function',S
print 'Learned Policy',Policy


Value Function {'whiteharbor': 11.51, 'winterfell': 58.5, 'dragonstone': 5.3075, 'dead-terminal': -50, 'alive-terminal': 100}
Learned Policy {'from_whiteharbor': 'land', 'from_dragonstone': 'dragon', 'from_winterfell': 'land'}


In [ ]:
#